## 1ª Lista de Exercícios - Deep Learning - Questão 2

![Questão 1](imgs/questoes/q2.png "Questão 2")

## Resolução

In [5]:
import numpy as np
import math

def randomizeArrayData(arr, variation=0.1):
    """ 
    Adiciona uma variação aleatoria em uma array de floats 
    """
    
    for i in range(len(arr)):
        arr[i] = arr[i] + (np.random.random()-0.5)*variation
        
    return arr

def criaDataset(n, variation=0.1):
    """ 
    Cria um conjunto de exemplos para a questão 1. Classificação de arestas em um cubo 
    """
    
    X = []
    Y = []
    
    # Crie n/8 exemplos para cada classe
    for i in range(math.ceil(n/8)):
        X.append(randomizeArrayData([0, 0, 0], variation)) 
        Y.append(0)
        X.append(randomizeArrayData([0, 0, 1], variation)) 
        Y.append(1)
        X.append(randomizeArrayData([0, 1, 0], variation)) 
        Y.append(2)
        X.append(randomizeArrayData([0, 1, 1], variation)) 
        Y.append(3)
        X.append(randomizeArrayData([1, 0, 0], variation)) 
        Y.append(4)
        X.append(randomizeArrayData([1, 0, 1], variation)) 
        Y.append(5)
        X.append(randomizeArrayData([1, 1, 0], variation)) 
        Y.append(6)
        X.append(randomizeArrayData([1, 1, 1], variation)) 
        Y.append(7)
        
    return X, Y

print("Criação de um dataset com 16 pontos e variação de +/- 20%:\n")
display(criaDataset(16, 0.2))

Criação de um dataset com 16 pontos e variação de +/- 20%:



([[0.09161826113661227, 0.08096895155806333, 0.0673314054643124],
  [0.0113760497154086, -0.008958776109398393, 0.9520910290397203],
  [-0.0007728094209140668, 1.0885474709665206, 0.03202434039390439],
  [0.026019273158738022, 1.0905755731202578, 0.905781572693711],
  [1.0844615618817437, -0.045155062461032805, 0.03309418343356514],
  [1.050105577830106, 0.03577607078369854, 1.0756023618760506],
  [0.974367210986845, 0.9652035488099325, 0.02819396504470395],
  [1.0405190710334131, 1.0249462016178847, 1.0564494138394127],
  [-0.09658736772080652, -0.09514597408118447, 0.014258047050399326],
  [0.023469130286267206, -0.03262718230954118, 1.0802703345695897],
  [-0.020563184757433618, 1.071777280965237, 0.08850096151005012],
  [-0.04576353271061698, 0.9584121279403071, 0.9830784122707754],
  [0.9276604507039399, -0.07193008595950848, -0.07724120193819262],
  [0.9106938730458676, 0.05973328303561301, 1.0504125913913651],
  [1.0427053529986223, 0.9547268463911157, 0.007634170996733469],
  [

In [31]:
import numpy as np
import math
import json
import matplotlib.pyplot as plt
from scipy.interpolate import spline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import pylab


# derivada da relu: 0 se menor que zero, e 1 se maior que zero


class Neuron:

    def __init__(self, inputs, lr = 0.01):
        
        # Os inputs pode ser um array de neuros ou um escalar
        if (isinstance(inputs, list)):
            self.weights = np.random.rand(len(inputs))
        else:
            self.weights = np.random.rand(inputs)
        
        self.bias = np.random.random() + 0.1
        self.inputs = inputs
        self.lr = lr if lr > 0 else 0.0001

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)


    def vj(self, x) :
        if (isinstance(self.inputs, list)):            
            signals = []            
            for i in range(len(self.inputs)):
                signals.append( self.inputs[i].forward(x))             
            return np.dot(self.weights, signals) + self.bias          
        else:
            return np.dot(self.weights, x) + self.bias
        
        
    def forward(self, x):
        return self.predict(self.activation( self.vj(x) ))


    def backward(self, x, y, soma=None): 

        somaPonderada = 0
        
        y_pred = self.forward(x)
        
        for idx in range(len(self.weights)):
            
            gradient = 0

            if (soma == None): 
                error = y - y_pred
                gradient = error * self.activation_derivative( self.vj(x) )
            else:
                gradient = self.activation_derivative( self.vj(x) ) * soma
       
            somaPonderada = somaPonderada + gradient*self.weights[idx]
            
            self.weights[idx] = self.weights[idx] + self.lr * x[idx] *  gradient
            
        if (isinstance(self.inputs, list)): 
            for i in range(len(self.inputs)):
                self.inputs[i].backward(x, y, soma=somaPonderada)
            
    def activation(self, x):
        return 0 if (x < 0) else x

    def activation_derivative(self, x):
        return 1 if (x >= 0) else 0
        
    def adjustWeights(self, x, error):            
        self.bias = self.bias + error*self.lr        
        for idx in range(len(self.weights)):
            self.weights[idx] = self.weights[idx] + error*self.lr*x[idx]

    def predict(self, out):
        """
        Função que discretiza a saída
        Parâmetro: y - saída do neurònio
        """
        if out < 0:
            return 0
        elif out > 7:
            return 7
        else:
            return round(out)
        
# TESTES
X, Y = criaDataset(200, 0.1)   

n_inputs = len(X[0])
        
layer1 = [ Neuron(n_inputs, lr=0.1) , Neuron(n_inputs, lr=0.1)]
    
layer2 = [ Neuron(layer1, lr=0.1)] 

network = layer2

#y_pred = network[0].forward(X[0])
#print(y_pred)

perceptron = network[0]

# perceptron = Neuron(len(X[0]))

print(perceptron)

for epoch in range(1000):
    erros_interacao = []
    for i in range(len(X)):
        y_pred = perceptron.forward(X[i])
        error = Y[i] - y_pred
        erros_interacao.append(error*error)
        perceptron.backward(X[i], Y[i])
    erro_medio = np.average(erros_interacao)
    #print(erro_medio)

# Preenche o array de saidas preditas
y_pred = []
for j in range(len(X)):
    y_pred.append(perceptron.forward(X[j]))

# Métricas de Avaliação
print('Matriz de Confusão:')
print(confusion_matrix(Y, y_pred))
print('F1 Score:')
print(classification_report(Y, y_pred))

<class '__main__.Neuron'>: {'weights': array([ 0.55930696,  0.10581092]), 'bias': 0.9283526622471622, 'inputs': [<__main__.Neuron object at 0x7fbac7cd01d0>, <__main__.Neuron object at 0x7fbac7cd0be0>], 'lr': 0.1}
Matriz de Confusão:
[[ 0 12  0  0  0  0  0 13]
 [ 0 25  0  0  0  0  0  0]
 [ 0 25  0  0  0  0  0  0]
 [ 0 25  0  0  0  0  0  0]
 [ 0 25  0  0  0  0  0  0]
 [ 0 25  0  0  0  0  0  0]
 [ 0 25  0  0  0  0  0  0]
 [ 0 25  0  0  0  0  0  0]]
F1 Score:
             precision    recall  f1-score   support

          0       0.00      0.00      0.00        25
          1       0.13      1.00      0.24        25
          2       0.00      0.00      0.00        25
          3       0.00      0.00      0.00        25
          4       0.00      0.00      0.00        25
          5       0.00      0.00      0.00        25
          6       0.00      0.00      0.00        25
          7       0.00      0.00      0.00        25

avg / total       0.02      0.12      0.03       200



/home/goldmine/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Criação do Perceptron

In [ ]:
import matplotlib.pyplot as plt
from scipy.interpolate import spline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import pylab

In [ ]:
def weight_init(num_inputs): 
    """
    Função que inicializa os pesos e bias aleatoriamente utilizando numpy
    Parâmetro: num_inputs - quantidade de entradas X
    Retorna: w,b - pesos e bias da rede inicializados
    """
    w = np.random.rand(num_inputs) - 0.5
    b =  np.random.random() - 0.5       
    return w,b

# Teste da função weight_init:
print(weight_init(5))

In [ ]:
def activation_func(func_type, z):
    """
    Função que implementa as funções de ativação mais comuns
    Parãmetros: func_type - uma string que contém a função de ativação desejada
                z - vetor com os valores de entrada X multiplicado pelos pesos
    Retorna: saída da função de ativação
    """
    if func_type == 'sigmoid':
        return 1/(1 + np.exp(-z))
    elif func_type == 'tanh':
        return math.sinh(z)/math.cosh(z)
    elif func_type == 'relu':
        return 0 if (z<0) else z
    elif func_type == 'linear':
        return z
    elif func_type == 'degrau':
        return 0 if (z < 0) else  1
    
def visualizeActivationFunc(z, func_type):
    pylab.rcParams['figure.figsize'] = (10.0, 5.0)
    z = np.arange(-5., 5., 0.2)
    func = []
    for i in range(len(z)):
        func.append(activation_func(func_type, z[i]))
    
    pylab.rcParams['figure.figsize'] = (5.0, 2.0)
    plt.plot(z,func)
    plt.title("Função: "+func_type)
    plt.xlabel('Entrada')
    plt.ylabel('Valores de Saída')
    plt.show()

# Testa as funções de ativação
visualizeActivationFunc(np.arange(-5., 5., 0.2), 'degrau')
visualizeActivationFunc(np.arange(-5., 5., 0.2), 'sigmoid')
visualizeActivationFunc(np.arange(-5., 5., 0.2), 'tanh')
visualizeActivationFunc(np.arange(-5., 5., 0.2), 'relu')
visualizeActivationFunc(np.arange(-5., 5., 0.2), 'linear')

In [ ]:
def forward(w,b,X):
    """
    Função que implementa a etapa forward propagate do neurônio
    Parâmetros: w - pesos
                b - bias
                X - entradas
    """
    z = np.dot(w,X)+b
    out = activation_func('relu', z)
    return out

# Teste: forward
x = (1, 1, 1)
w,b = weight_init(3)
print("X: ", x)
print("Pesos: ", w)
print("Bias: ", b)
print("Y: ", forward(w,b,x))

In [ ]:
def predict(out):
    """
    Função que discretiza a saída
    Parâmetro: y - saída do neurònio
    """
    if out < 0:
        return 0
    elif out > 7:
        return 7
    else:
        return round(out)

# Teste: predict
print ("Entrada=1.1 -> Saida=", predict(1.1))
print ("Entrada=4.5 -> Saida=", predict(4.5))
print ("Entrada=70.0 -> Saida=", predict(70.0))

In [ ]:
def perceptron(x, y, num_interaction, learning_rate):
    """
    Função que implementa o loop do treinamento 
    Parâmetros: x - entrada da rede 
                y - rótulos/labels
                num_interaction - quantidade de interações desejada para a rede convergir
                learning_rate - taxa de aprendizado para cálculo do erro
    """
    training_interation = []
    training_erro = []
    
    #Passo 1 - Inicie os pesos e bias (~1 linha)
    w,b = weight_init(len(x[0]))
    #Passo 2 - Loop por X interações
    for i in range(num_interaction):
        
        # Ajuda no calculo do erro médio quadrado
        erros_interacao = []
        
        for j in range(len(x)): # para cada exemplo
            
            # Passo 3 -  calcule a saída do neurônio (~1 linha)
            y_calc = predict(forward(w,b,x[j]))
            
            # Passo 4 - calcule o erro entre a saída obtida e a saída desejada nos rótulos/labels (~1 linha)
            erro = y[j] - y_calc  
            
            # Adiciona o erro quadrado dessa instancia
            erros_interacao.append(erro*erro)
            
            # Ajusta os pesos e bias
            for idx in range(len(w)):
                w[idx] = w[idx] + erro*learning_rate*x[j][idx]            

            b = b + erro*learning_rate
        
        # Calcula o erro médio dessa interação
        erro_medio = np.average(erros_interacao)
        
        training_interation.append(i)
        training_erro.append(erro_medio) 
        
        # Apenas para garantir que pare quando o erro atingir um valor limite mínimo
        if (erro_medio < 0.00001):
            print("Finalizado na interação %d visto que o erro médio já se tornou muito pequeno %f" % (i, erro_medio))
            break;
    
    # Cria um grafico com os dados sobre o erro de treinamento
    pylab.rcParams['figure.figsize'] = (15.0, 10.0)
    plt.plot(training_interation, training_erro)
    plt.xlabel('Interação')
    plt.ylabel('Erro médio quadrado')
    plt.show()
    
    return w,b

def validar(w,b,x,y):

    # Preenche o array de saidas preditas
    y_pred = []
    for j in range(len(x)):
        y_pred.append(predict(forward(w,b,x[j])))
       
    # Métricas de Avaliação
    print('Matriz de Confusão:')
    print(confusion_matrix(y, y_pred))
    print('F1 Score:')
    print(classification_report(y, y_pred))


### Criação dos Datasets de Treinamento e Validação

In [ ]:
X_train, Y_train = criaDataset(800, 0.1)
X_val, Y_val = criaDataset(200, 0.1)

### Trainamento do Perceptron

In [ ]:
w, b = perceptron(X_train, Y_train, 1000, 0.001)
print("Pesos Aprendidos:", w, "\nBias Aprendido:", b)

## Resultados

#### Conjunto de Treinamento

In [ ]:
validar(w, b, X_train, Y_train)

#### Conjunto de Validação

In [ ]:
validar(w, b, X_val, Y_val)